In [ ]:
import torch
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
import optiprop
import numpy as np
import os

In [ ]:
# Configuration
# - device: 'cpu' or 'cuda' (if you have a GPU and CUDA-enabled PyTorch installed)
# - dtype: float32 is memory-friendly; float64 increases precision
device = 'cpu'
dtype = torch.float32

# Near-field grid definition
# - NearField_pixel_size: sampling interval (m) on the near-field plane
# - NearField_field_Lx/Ly: physical size (m) of the simulated field in X and Y
NearField_pixel_size = 500e-9
NearField_field_Lx, NearField_field_Ly = 500e-6, 500e-6

# Create the near-field grid on which phase elements will be defined
Input_field = optiprop.NearField(
    pixel_size=NearField_pixel_size,
    field_Lx=NearField_field_Lx,
    field_Ly=NearField_field_Ly,
    dtype=dtype,
    device=device,
)

# Pretty print grid details (size, pixel pitch, device, etc.)
Input_field.rich_print()

In [ ]:
# Metalens phase definition (Binary2 polynomial phase)
# - wavelength: design wavelength (m) used for illumination later
# - lens_diameter: aperture diameter of the meta-lens (m)
# - Binary2: polynomial coefficients for phase profile (example values)
wavelength = 0.940e-6
lens_diameter = 500e-6
Binary2 = [-1.6768e+03, 120.4642, -29.2946, 7.1850]

# Instantiate the Binary2Phase element on the near-field grid
binary2_element = optiprop.Binary2Phase(Input_field)

# Compute complex field U0 of the metalens phase, with a circular aperture
binary2_element.calculate_phase(
    binary2=Binary2,
    lens_diameter=lens_diameter,
    lens_center=[0, 0],
    aperture_type='circle',
    aperture_size=[lens_diameter],
)

# Print parameters and draw phase/amplitude/field for inspection
binary2_element.rich_print()
binary2_element.draw(
    cmap='turbo',
    fontsize=24,
    xlim=None,
    ylim=None,
    selected_field='all',
    dark_style=True,
    show=True,
)


In [ ]:
# Visualize the metalens phase at multiple scales
# 1) Full field view
binary2_element.draw(
    cmap='turbo',
    fontsize=24,
    xlim=None,
    ylim=None,
    dark_style=True,
    selected_field='all',
    show=True,
)

# 2) Zoomed-in view around the center (±100 µm)
binary2_element.draw(
    cmap='turbo',
    fontsize=24,
    xlim=[-100, 100],
    ylim=[-100, 100],
    selected_field='all',
    dark_style=True,
    show=True,
)

# 3) Further zoom around the center (±50 µm)
binary2_element.draw(
    cmap='turbo',
    fontsize=24,
    xlim=[-50, 50],
    ylim=[-50, 50],
    selected_field='all',
    dark_style=True,
    show=True,
)

In [ ]:
# Define a plane-wave incident field (tilt can be introduced via incident_angle)
incident_source = optiprop.IncidentField(Input_field)
incident_source.calculate_phase(
    incident_angle=[0, 0],  # [theta_x, theta_y] in degrees
    n=1,                    # refractive index in object space
    design_lambda=wavelength, 
    lens_center=[0, 0],
    aperture_type='circle',
    aperture_size=[lens_diameter],
)

# Print and visualize the incident phase
incident_source.rich_print()
incident_source.draw(
    cmap='turbo',
    fontsize=24,
    xlim=None,
    ylim=None,
    selected_field='all',
    dark_style=True,
    show=True,
)

In [ ]:
# Combine the illumination (incident plane wave) with the metalens phase
# U_total is the complex field at the near-field plane that will be propagated
U_total = incident_source.U0 * binary2_element.U0

# Register the field on the NearField helper (for convenient plotting)
Input_field.input_field(U_total)

# Visualize the resulting field, amplitude and phase
Input_field.draw(
    U_total,
    cmap='turbo',
    fontsize=24,
    xlim=None,
    ylim=None,
    selected_field='all',
    dark_style=True,
    show=True,
)


In [ ]:
# Define the target focal length (propagation distance for far field observation)
focal_length = 2000e-6

# Create a Fresnel propagator
# - propagation_wavelength: effective wavelength in the medium (m)
# - propagation_distance: distance between near-field plane and observation plane (m)
# - n: refractive index (1.0 for air)
propagator = optiprop.FresnelPropagation(
    propagation_wavelength=wavelength,
    propagation_distance=focal_length,
    n=1,
    device=device,
    dtype=dtype,
)

# Feed the combined near-field into the propagator
propagator.set_input_field(
    u_in=U_total,
    pixel_size=NearField_pixel_size,
)

# Inspect the input field settings and visualization
propagator.show_input_U(
    cmap='turbo',
    fontsize=24,
    xlim=None,
    ylim=None,
    dark_style=True,
    show=True,
)


In [ ]:
# Optionally redefine the output sampling on the observation plane
# This changes the pixel size and field-of-view for the rendered output
propagator.set_output_field(
    output_pixel_size=0.5e-6,
)

# Before propagation, show the current output plane configuration
propagator.show_output_U(
    cmap='turbo',
    fontsize=24,
    xlim=None,
    ylim=None,
    selected_field='all',
    dark_style=True,
    show=True,
)


In [ ]:
# Run Fresnel propagation to the focal plane and visualize complex field
propagator.propagate()
propagator.show_output_U(
    cmap='turbo',
    fontsize=24,
    xlim=[-25, 25],
    ylim=[-25, 25],
    selected_field='all',
    dark_style=True,
    show=True,
)

In [ ]:
# Show intensity distribution at the focal plane (linear scale)
propagator.show_intensity(
    cmap='CMRmap',
    fontsize=24,
    xlim=[-25, 25],
    ylim=[-25, 25],
    clim=None,
    unit='µm',
    norm='linear',
    dark_style=True,
    show=True,
)

In [ ]:
# Scan along z to visualize beam evolution around the focus
# Here we sample 51 planes from 1.6 mm to 2.4 mm
z_axis = torch.linspace(1600e-6, 2400e-6, 51)
propagator.propagate_xz(
    z_range=z_axis,
)

In [ ]:
# Show XZ intensity map (linear scale)
propagator.show_xz_intensity(
    cmap='CMRmap',
    fontsize=16,
    xlim=[-50, 50],
    zlim=None,
    clim=None,
    unit='µm',
    norm='linear',
    dark_style=True,
    title=None,
    show=True,
)

# Show XZ intensity map (log scale) to reveal sidelobes and weak features
propagator.show_xz_intensity(
    cmap='CMRmap',
    fontsize=16,
    xlim=[-50, 50],
    zlim=None,
    clim=[1, 1e5],
    unit='µm',
    norm='log',
    dark_style=True,
    title=None,
    show=True,
)